In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import *
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials

workspace = os.path.dirname(os.path.realpath('__file__'))
utils.DEBUG = True  # print debug messages. Calls are successful as long as no exception is thrown.

In [4]:
credentials.login('bWEPDTmZIFiSJbYw15FXevi0FZ3bp2Tk')

In [5]:
# Upload code
train_code = os.path.join(workspace, 'example.zip')
# eval_code = os.path.join(workspace, 'demo/eval_script.zip')
File.upload({train_code: 'ray_profiler_train_scripts.zip'})

Running request from dev: 35.237.138.197 80 storage start_file_upload_session
POST data {"paths": ["ray_profiler_train_scripts.zip"], "hashes": ["62be01b2d0fbe91ca152cdda0aa8d0a0"], "token": "bWEPDTmZIFiSJbYw15FXevi0FZ3bp2Tk"}
[('/Users/baljitsingh/Desktop/Capstone-F20-latest/acaisdk/acaisdk/example/ray-profiler-example/example.zip', 'ray_profiler_train_scripts.zip')]
r =  <Response [200]>
Uploaded /Users/baljitsingh/Desktop/Capstone-F20-latest/acaisdk/acaisdk/example/ray-profiler-example/example.zip to ray_profiler_train_scripts.zip
Running request from dev: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 8730, "token": "bWEPDTmZIFiSJbYw15FXevi0FZ3bp2Tk"}
Running request from dev: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 8730, "token": "bWEPDTmZIFiSJbYw15FXevi0FZ3bp2Tk"}
Running request from dev: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 8730, "token": "bWEPDTmZIFiSJbYw15FXevi0FZ3bp2Tk"}
R

[('/Users/baljitsingh/Desktop/Capstone-F20-latest/acaisdk/acaisdk/example/ray-profiler-example/example.zip',
  'ray_profiler_train_scripts.zip:1')]

In [6]:
# Run a profiling job
job_setting = {
    "command_template": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt",
    "container_image": "rayproject/autoscaler",
    'input_file_set': 'ray_test_fs', # not used since this example does not need an input dataset
    'code': 'ray_profiler_train_scripts.zip',
    'name': 'ray_profiling_test',
    'framework': 'ray',
    'nnode_space': [1],
    'memory_space': [512]
}

result = {}
j = ProfilingJob().with_attributes(job_setting)

In [7]:
# this will populate the result dict with the response, you can also get the response by assigning the return value to a variable
j.profile(result)

Running request from dev: 35.237.138.197 80 profiler register_template
POST data {"name": "ray_profiling_test", "input_file_set": "ray_test_fs", "code": "ray_profiler_train_scripts.zip", "command_template": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt", "container_image": "rayproject/autoscaler", "memory_space": [512], "nnode_space": [1], "framework": "ray", "token": "bWEPDTmZIFiSJbYw15FXevi0FZ3bp2Tk"}
{'id': 43, 'name': 'ray_profiling_test', 'code': 'ray_profiler_train_scripts.zip', 'allOptionsSer': '[]', 'state': 'submitted', 'input_file_set': 'ray_test_fs', 'command_template': 'ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python exa

{'id': 43,
 'name': 'ray_profiling_test',
 'code': 'ray_profiler_train_scripts.zip',
 'allOptionsSer': '[]',
 'state': 'submitted',
 'input_file_set': 'ray_test_fs',
 'command_template': 'ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt',
 'memory_space': [512],
 'cpu_space': [0.5, 1.0],
 'nnode_space': [1],
 'user_id': 3446,
 'project_id': 'integration_test_dev',
 'container_image': 'rayproject/autoscaler',
 'model_name': None,
 'framework': 'ray',
 'model_id': None,
 'spawned_jobs': 0}